## Либо переделывать под остальные теги кроме w (i, b), либо брать тексты без них

In [1]:
# https://pep8.ru/doc/dive-into-python-3/14.html
import os
import urllib3
import xml
import sys
import xml.etree.ElementTree as ET
import numpy as np
from lxml import html


import re

## Перебор по файлам из всех папок

In [2]:
# Перебор по всем файлам!

path = "Data/TEXTS/"

folders = os.listdir(path)
folders
for folder in folders:
    print(folder)
    files = os.listdir(path + str(folder))
    print(len(files))

speech
65
blogs_2013
24
fiction
32
science
85
public
349


## Достаем все теги из всех фалов!

In [3]:
import re
retag = re.compile(r'\S+')

def findtag(tag):
    
    strtag = str(tag)
    m = retag.finditer(strtag)
    strings = [i.span() for i in m]

    match = strtag[strings[1][0] : strings[1][1]]    
    
    return match

In [4]:
def clearword(word):
    accent = word.find('`') # Смотрим позицию ударения
    if accent != -1: # Избавляемся, если есть
        word = word[:accent] + word[accent + 1:]
        word = clearword(word)

    return word
clearword('з````````д``о`р`о`в`о```')

'здорово'

In [5]:
reg = re.compile(r'\S+')
def clearpunct(punct):
    wat = reg.findall(punct)
    clrpu = ''
    for i in wat:
        clrpu += i    
   
    return clrpu
clearpunct(' , -- " asdawr ')

',--"asdawr'

In [59]:
# слов: list = 959604 -> set = 133999
# puncts: list = 1049392 -> set = 1359
# taggs: list = 959604 -> set = 115

path = "Data/TEXTS/"
folders = os.listdir(path)

General = []


# По папкам
for folder in folders:
    files = os.listdir(path + str(folder))
    print('Folder, num of files:', folder, len(files))
    
    # По файлам
    for file in files:
        
        filepath = path + str(folder) + '/' + str(file)
        
        tree = html.parse(filepath)
        root = tree.getroot()
        paragraphs = tree.findall('.//p')

        # По параграфам
        for paragraph in paragraphs:
                        
                
            # По тегам внутри <p>
            for sentence in paragraph:
                
                
                # Исключим плохие: если парсим не <se> и если в <se> есть <sub>/<sup>
                match = findtag(sentence)
                if match != 'se':
                    continue
                
                list_of_tags = []
                # Перебор по всем тегам
                for tag in sentence:
                    match = findtag(tag)
                    list_of_tags.append(match)
                set_of_tags = set(list_of_tags)
                
                if 'sub' in set_of_tags or 'sup' in set_of_tags:
                    continue                    
                    
                
                
                # Достаем нужную инфу из оставшихся...
                                
                alltexts = sentence.xpath('.//text()') # Все тексты внутри <se>
                words = sentence.xpath('.//w/text()') # Тексты внутри тегов <w> в нашем <se>

                # Если список слов пустой - континью
                if len(words) == 0:
                    continue
                    
                tags = sentence.findall('.//ana[@gr]') # Все теги для слов
                wordtags = sentence.findall('.//w')

                
                len_all = len(alltexts)

                # ИДЕМ ПО ТЕГАМ ВОРД!
                
                parsesent = []
                count = 0 # main count!!!!
                for tag in alltexts:
                    
                    
                    # Сравниваем слова и теги
                    if tag == words[count]:

                        cwrdt = wordtags[count].findall('.//ana[@gr]')
                        if cwrdt == 1:
                            wordtag = tags[count].attrib['gr'].split(',')
                            wordlem = tags[count].attrib['lex'].encode('latin1').decode('cp1251')
                        else:
                            wordtag = cwrdt[0].attrib['gr'].split(',')
                            wordlem = cwrdt[0].attrib['lex'].encode('latin1').decode('cp1251')  

                        word = tag.encode('latin1').decode('cp1251')
                
                    
                        if len(words) - 1 > count:
                            count += 1
                            
                        parsetag = [clearword(word), wordlem, wordtag]
                        parsesent.append(parsetag)
                        
                    else:
                        
                        if len(clearpunct(tag)) != 0:
                            parsetag = [clearpunct(tag), clearpunct(tag), ['PUNCT']]
                            parsesent.append(parsetag)
                    
                General.append(parsesent)
                    
                #print(parsesent)


Folder, num of files: speech 65
Folder, num of files: blogs_2013 24
Folder, num of files: fiction 32
Folder, num of files: science 85
Folder, num of files: public 349


In [60]:
General[-30:-5]

[[['Любимый', 'любимый', ['S', 'm', 'anim=sg', 'nom']],
  ['мой', 'мой', ['A-PRO=m', 'sg', 'nom']],
  ['мне', 'я', ['S-PRO', 'sg', '1p=dat']],
  ['очень', 'очень', ['ADV']],
  ['помог', 'помочь', ['V', 'pf', 'intr', 'act=m', 'sg', 'praet', 'indic']],
  [',', ',', ['PUNCT']],
  ['и', 'и', ['CONJ']],
  ['не', 'не', ['PART']],
  ['только', 'только', ['PART']],
  ['тем', 'то', ['S-PRO', 'n', 'sg=ins']],
  [',', ',', ['PUNCT']],
  ['что', 'что', ['CONJ']],
  ['убеждал',
   'убеждать',
   ['V', 'ipf', 'tran=m', 'sg', 'act', 'praet', 'indic']],
  ['читать', 'читать', ['V', 'ipf', 'tran=inf', 'act']],
  ['Шекспира', 'Шекспир', ['S', 'famn', 'm', 'anim=sg', 'acc']],
  ['в', 'в', ['PR']],
  ['подлиннике', 'подлинник', ['S', 'm', 'inan=sg', 'loc']],
  [',', ',', ['PUNCT']],
  ['или', 'или', ['CONJ']],
  ['тем', 'то', ['S-PRO', 'n', 'sg=ins']],
  [',', ',', ['PUNCT']],
  ['что', 'что', ['CONJ']],
  ['регулярно', 'регулярно', ['ADV']],
  ['беседовал',
   'беседовать',
   ['V', 'ipf', 'intr', 'act=m

## СОХРАНЕНИЕ - ОТКРЫТИЕ файла

#### Сохранение - попроще, в каждой строке по предложению, каждый четный элемент - слово, следующий за ним - тег

In [98]:
import csv

with open('Data/Tags/Parsed.csv', 'w', newline='') as csvfile:
    csv_writer = csv.writer(csvfile)
    
    for item in General:
        for i in item:
            csv_writer.writerow(i)
        csv_writer.writerow('')

#### Пробуем правильно распаковать

In [100]:
# Этим способом получилось сделать как в оригинале

Dataset = []
with open('Data/Tags/Parsed.csv', newline='') as csvfile:
    csv_reader = csv.reader(csvfile, delimiter=',')
   
    data = []
    for row in csv_reader:
        
        if len(row) == 0:
            Dataset.append(data)
            data = []
        else:
            tag = [row[0], row[1]]
            toplst = [i[1:-1] for i in row[2][1:-1].split(', ')]
            tag.append(toplst)
            data.append(tag)

                
print('end')

end


In [101]:
Dataset[100:130]

[[['Нет', 'нет', ['PART']], ['.', '.', ['PUNCT']]],
 [['Один', 'один', ['ANUM=m', 'nom']],
  ['только', 'только', ['PART']],
  ['кусочек', 'кусочек', ['S', 'm', 'inan=sg', 'nom']],
  ['холодный', 'холодный', ['A=m', 'sg', 'nom', 'plen']],
  ['остался',
   'остаться',
   ['V', 'pf', 'intr', 'med=m', 'sg', 'praet', 'indic']],
  ['.', '.', ['PUNCT']]],
 [['В', 'В', ['INIT=abbr']], ['..', '..', ['PUNCT']]],
 [['Давайте', 'давайте', ['PART']],
  ['погрею', 'погреть', ['V', 'pf', 'tran=sg', 'act', 'fut', '1p', 'indic']],
  ['.', '.', ['PUNCT']]],
 [['Б', 'Б', ['INIT=abbr']], ['..', '..', ['PUNCT']]],
 [['Нет', 'нет', ['PART']],
  ['/', '/', ['PUNCT']],
  ['я', 'я', ['S-PRO', 'sg', '1p=nom']],
  ['больше', 'больше', ['ADV']],
  ['не', 'не', ['PART']],
  ['хочу', 'хотеть', ['V', 'ipf', 'tran', 'act=sg', 'praes', '1p', 'indic']],
  ['.', '.', ['PUNCT']]],
 [['Д', 'Д', ['INIT=abbr']], ['..', '..', ['PUNCT']]],
 [['А', 'а', ['CONJ']],
  ['кашу', 'каша', ['S', 'f', 'inan=sg', 'acc']],
  ['?', '?',